In [15]:
import matplotlib.pyplot as plt
import cv2
import glob, pylab, pandas as pd
import numpy as np
%matplotlib inline

import time
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras

from keras.applications import Xception

from keras import callbacks

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Conv3D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.models import Model


In [16]:
df_detailed = pd.read_csv('./stage_2_detailed_class_info.csv')
df_detailed['patientId'] =   df_detailed['patientId'].astype(str) + '.png' 

In [17]:
image_gen = ImageDataGenerator(rotation_range = 30,
                              width_shift_range= 0.1, 
                              height_shift_range= 0.1,
                              rescale= 1/255,
                              shear_range=0.2,
                              zoom_range= 0.2,
                              horizontal_flip= True,
                              fill_mode='nearest',
                               validation_split =0.2
                              )

In [18]:
train_image_gen = image_gen.flow_from_dataframe(dataframe = df_detailed,
                             directory= './train',
                             x_col = 'patientId',
                             y_col = 'class',
                              target_size=(256,256),
                              color_mode='rgb',
                              classes= None,
                              class_mode='categorical',
                              batch_size=16,
                              shuffle=True,
                              subset = 'training'
                             )

Found 24015 validated image filenames belonging to 3 classes.


In [19]:
test_image_gen = image_gen.flow_from_dataframe(dataframe = df_detailed,
                             directory= './train',
                             x_col = 'patientId',
                             y_col = 'class',
                              target_size=(256,256),
                              color_mode='rgb',
                              classes= None,
                              class_mode='categorical',
                              batch_size=16,
                              shuffle=True,
                              subset = 'validation'
                             )

Found 6003 validated image filenames belonging to 3 classes.


In [20]:
orig_net = Xception(include_top=False, weights='imagenet', input_shape=(256,256,3)) 


filters = GlobalAveragePooling2D()(orig_net.output) 


classifiers = Dense(3, activation='softmax', bias_initializer='ones')(filters)  

model = Model(inputs=orig_net.inputs, outputs=classifiers)  

model.compile(loss='categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [21]:
from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("Xception_2.h5", monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto',
                             period=1)

early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=20, 
                      verbose=1, 
                      mode='auto')

results = model.fit_generator(train_image_gen, epochs =150, 
                              steps_per_epoch=800,
                             validation_data=test_image_gen,  
                              callbacks=[checkpoint,early])

Epoch 1/150


INFO:plaidml:Analyzing Ops: 1849 of 5836 operations complete
INFO:plaidml:Analyzing Ops: 2883 of 5836 operations complete


 15/800 [..............................] - ETA: 44:50 - loss: nan - acc: 0.3125

INFO:plaidml:Analyzing Ops: 2003 of 5836 operations complete
INFO:plaidml:Analyzing Ops: 2958 of 5836 operations complete


800/800 [==============================] - 1877s 2s/step - loss: nan - acc: 0.2945 - val_loss: nan - val_acc: 0.4233

Epoch 00001: val_acc improved from -inf to 0.42329, saving model to Xception_2.h5
Epoch 2/150
800/800 [==============================] - 1816s 2s/step - loss: nan - acc: 0.2873 - val_loss: nan - val_acc: 0.4233

Epoch 00002: val_acc did not improve from 0.42329
Epoch 3/150
800/800 [==============================] - 1980s 2s/step - loss: nan - acc: 0.2892 - val_loss: nan - val_acc: 0.4233

Epoch 00003: val_acc did not improve from 0.42329
Epoch 4/150
800/800 [==============================] - 1849s 2s/step - loss: nan - acc: 0.2898 - val_loss: nan - val_acc: 0.4233

Epoch 00004: val_acc did not improve from 0.42329
Epoch 5/150
800/800 [==============================] - 1696s 2s/step - loss: nan - acc: 0.2899 - val_loss: nan - val_acc: 0.4233

Epoch 00005: val_acc improved from 0.42329 to 0.42329, saving model to Xception_2.h5
Epoch 6/150
800/800 [========================